# **DATATHON 2023: NTT-DATA CHALLENGE**

## **Requirements**:

In [30]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## **Main Program**

Read dataset

In [ ]:
df = pd.read_excel('../assets/consumo_material_clean.xlsx')
df.head()

Some preprocessing

In [ ]:
# Drop rows with NaN values
df = df.dropna()

# Split "ORIGEN" into "REGION", "HOSPITAL" and "DEPARTAMENTO"
df['ORIGEN'] = df['ORIGEN'].str.replace('--', '-')
df[['REGION', 'HOSPITAL', 'DEPARTAMENTO']] = df['ORIGEN'].str.split('-', expand=True)
df = df.drop(["ORIGEN"], axis=1)

# Categorical variables
categorical = ['CODIGO', 'PRODUCTO', 'NUMERO', 'REFERENCIA', 'TIPOCOMPRA', 'REGION', 'HOSPITAL', 'DEPARTAMENTO', 'TGL']
df[categorical] = df[categorical].astype('category')

# Numeric variables
numerical_int = ['CANTIDADCOMPRA', 'UNIDADESCONSUMOCONTENIDAS']
numerical_float = ['PRECIO', 'IMPORTELINEA']
df[numerical_float] = df[numerical_float].astype('float')

# Date format
df['FECHAPEDIDO'] = pd.to_datetime(df['FECHAPEDIDO'], format='%d/%m/%y')
df['MES'] = df['FECHAPEDIDO'].dt.month
df['AÑO'] = df['FECHAPEDIDO'].dt.year
df = df.drop('FECHAPEDIDO', axis=1)

# Create new variable
df['PRECIOUNIDAD'] = df['IMPORTELINEA'] / df['CANTIDADCOMPRA']

df.to_csv('../assets/preprocessed_df.csv', index=False)

df.head()

In [ ]:
años = df['AÑO'].unique()
meses = df['MES'].unique()
productos = df['PRODUCTO'].unique()

# Crear todas las combinaciones posibles de año, mes y producto
todas_las_combinaciones = pd.DataFrame(list(itertools.product(años, meses, productos)), columns=['AÑO', 'MES', 'PRODUCTO'])

# Agrupar por año, mes y producto y realizar las operaciones de agregación
df_agrupado = df.groupby(['AÑO', 'MES', 'PRODUCTO']).agg({'CANTIDADCOMPRA': 'sum'}).reset_index()

# Combinar el DataFrame de todas las combinaciones con el DataFrame agrupado
# Usar un merge para asegurarse de que todas las combinaciones estén presentes
resultado_final = pd.merge(todas_las_combinaciones, df_agrupado, on=['AÑO', 'MES', 'PRODUCTO'], how='left')

# Rellenar los valores faltantes con 0 (o cualquier otro valor que sea apropiado)
resultado_final['CANTIDADCOMPRA'] = resultado_final['CANTIDADCOMPRA'].fillna(0)

resultado_final.to_csv('../assets/cantidadcompra_combinations.csv', index=False)


In [ ]:
años = df['AÑO'].unique()
meses = df['MES'].unique()
productos = df['PRODUCTO'].unique()

# Crear todas las combinaciones posibles de año, mes y producto
todas_las_combinaciones = pd.DataFrame(list(itertools.product(años, meses, productos)), columns=['AÑO', 'MES', 'PRODUCTO'])

# Agrupar por año, mes y producto y realizar las operaciones de agregación
df_agrupado = df.groupby(['AÑO', 'MES', 'PRODUCTO']).agg({'PRECIOUNIDAD': 'mean'}).reset_index()

# Combinar el DataFrame de todas las combinaciones con el DataFrame agrupado
# Usar un merge para asegurarse de que todas las combinaciones estén presentes
resultado_final = pd.merge(todas_las_combinaciones, df_agrupado, on=['AÑO', 'MES', 'PRODUCTO'], how='left')

# Rellenar los valores faltantes con 0 (o cualquier otro valor que sea apropiado)
resultado_final['PRECIOUNIDAD'] = resultado_final['PRECIOUNIDAD'].fillna(0)

resultado_final.to_csv('../assets/preciounidad_combinations.csv', index=False)


Split train and test datasets

In [ ]:
split_year = 2023
train = new_df.loc[new_df['AÑO'] < split_year]
test = new_df.loc[new_df['AÑO'] >= split_year]

In [ ]:
train.to_csv('../assets/train.csv', index=False)
test.to_csv('../assets/test.csv', index=False)

ARIMA TO PREDICT CANTIDADCOMPRA

In [ ]:
from pmdarima import auto_arima
from pmdarima.arima.utils import ndiffs
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Carga de datos
data = pd.read_csv('../assets/cantidadcompra_combinations.csv')

# Agrupación de datos por producto y mes
grouped_data = data.groupby(['PRODUCTO', 'AÑO', 'MES']).agg({'CANTIDADCOMPRA': 'sum'}).reset_index()

# Inicializar un diccionario para almacenar los modelos
models = {}

# Función para verificar la estacionariedad
def check_stationarity(ts):
    return ndiffs(ts, test='adf') > 0

# Iterar sobre cada producto
for product_name in grouped_data['PRODUCTO'].unique():
    product_data = grouped_data[grouped_data['PRODUCTO'] == product_name].copy()

    # Crear un índice de tiempo combinando año y mes
    product_data.loc[:, 'Fecha'] = pd.to_datetime(product_data['AÑO'].astype(str) + '-' + product_data['MES'].astype(str)) + MonthEnd(1)
    product_series = product_data.set_index('Fecha')['CANTIDADCOMPRA']

    # Dividir en conjunto de entrenamiento y prueba
    train = product_series[:'2022-12-31']
    test = product_series['2023-01-31':'2023-10-31']

    try:
        # Aplicar Auto ARIMA
        stationarity = check_stationarity(train)
        model = auto_arima(train, seasonal=stationarity, m=12 if stationarity else 0, trace=True, error_action='ignore', suppress_warnings=True)
        models[product_name] = model
    except Exception as e:
        print(f"Error al modelar el producto {product_name}: {e}")

# Preparar los datos de prueba y alinearlos con las predicciones
test_data = data[(data['AÑO'] == 2023) & (data['MES'] <= 10)].copy()

# Realizar predicciones y comparar con los datos de prueba
for product_name, model in models.items():
    if product_name in grouped_data['PRODUCTO'].unique():
        # Realizar la predicción
        forecast = model.predict(n_periods=10)

        # Insertar predicciones en test_data
        for i, month in enumerate(range(1, 11)):
            test_data.loc[(test_data['PRODUCTO'] == product_name) & (test_data['MES'] == month), 'predicted'] = forecast[i]

        # Extraer valores reales para el gráfico
        actual_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['CANTIDADCOMPRA']

        # Gráfico de las predicciones para cada producto
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, 11), actual_values, label='Valores Reales', color='blue')
        plt.plot(range(1, 11), forecast, label='Predicciones', color='red', linestyle='--')
        plt.title(f'Comparación de Valores Reales y Predicciones para {product_name} en 2023')
        plt.xlabel('Meses del 2023')
        plt.ylabel('Cantidad Comprada')
        plt.legend()
        plt.show()

from sklearn.metrics import r2_score

# Preparar listas para almacenar todos los valores reales y predichos
all_actual_values = []
all_predicted_values = []

for product_name in grouped_data['PRODUCTO'].unique():
    if product_name in test_data['PRODUCTO'].unique():
        actual_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['CANTIDADCOMPRA']
        predicted_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['predicted']

        # Añadir valores reales y predichos a las listas generales
        all_actual_values.extend(actual_values)
        all_predicted_values.extend(predicted_values)

# Calcular R^2 para todos los productos en conjunto
r2_general = r2_score(all_actual_values, all_predicted_values)
print(f"R^2 general: {r2_general}")


# Guardar el conjunto de datos de prueba con las predicciones en un archivo CSV
test_data.to_csv('../assets/cantidadcompra_predicted.csv', index=False)

ARIMA TO PREDICT PRECIOUNIDAD

In [ ]:
from pmdarima import auto_arima
from pmdarima.arima.utils import ndiffs
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Carga de datos
data = pd.read_csv('../assets/preciounidad_combinations.csv')

# Agrupación de datos por producto y mes
grouped_data = data.groupby(['PRODUCTO', 'AÑO', 'MES']).agg({'PRECIOUNIDAD': 'mean'}).reset_index()

# Inicializar un diccionario para almacenar los modelos
models = {}

# Función para verificar la estacionariedad
def check_stationarity(ts):
    return ndiffs(ts, test='adf') > 0

# Iterar sobre cada producto
for product_name in grouped_data['PRODUCTO'].unique():
    product_data = grouped_data[grouped_data['PRODUCTO'] == product_name].copy()

    # Crear un índice de tiempo combinando año y mes
    product_data.loc[:, 'Fecha'] = pd.to_datetime(product_data['AÑO'].astype(str) + '-' + product_data['MES'].astype(str)) + MonthEnd(1)
    product_series = product_data.set_index('Fecha')['PRECIOUNIDAD']

    # Dividir en conjunto de entrenamiento y prueba
    train = product_series[:'2022-12-31']
    test = product_series['2023-01-31':'2023-10-31']

    try:
        # Aplicar Auto ARIMA
        stationarity = check_stationarity(train)
        model = auto_arima(train, seasonal=stationarity, m=12 if stationarity else 0, trace=True, error_action='ignore', suppress_warnings=True)
        models[product_name] = model
    except Exception as e:
        print(f"Error al modelar el producto {product_name}: {e}")

# Preparar los datos de prueba y alinearlos con las predicciones
test_data = data[(data['AÑO'] == 2023) & (data['MES'] <= 10)].copy()

# Realizar predicciones y comparar con los datos de prueba
for product_name, model in models.items():
    if product_name in grouped_data['PRODUCTO'].unique():
        # Realizar la predicción
        forecast = model.predict(n_periods=10)

        # Insertar predicciones en test_data
        for i, month in enumerate(range(1, 11)):
            test_data.loc[(test_data['PRODUCTO'] == product_name) & (test_data['MES'] == month), 'predicted'] = forecast[i]

        # Extraer valores reales para el gráfico
        actual_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['PRECIOUNIDAD']

        # Gráfico de las predicciones para cada producto
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, 11), actual_values, label='Valores Reales', color='blue')
        plt.plot(range(1, 11), forecast, label='Predicciones', color='red', linestyle='--')
        plt.title(f'Comparación de Valores Reales y Predicciones para {product_name} en 2023')
        plt.xlabel('Meses del 2023')
        plt.ylabel('Cantidad Comprada')
        plt.legend()
        plt.show()

from sklearn.metrics import r2_score

# Preparar listas para almacenar todos los valores reales y predichos
all_actual_values = []
all_predicted_values = []

for product_name in grouped_data['PRODUCTO'].unique():
    if product_name in test_data['PRODUCTO'].unique():
        actual_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['PRECIOUNIDAD']
        predicted_values = test_data[(test_data['PRODUCTO'] == product_name) & (test_data['AÑO'] == 2023) & (test_data['MES'] <= 10)]['predicted']

        # Añadir valores reales y predichos a las listas generales
        all_actual_values.extend(actual_values)
        all_predicted_values.extend(predicted_values)

# Calcular R^2 para todos los productos en conjunto
r2_general = r2_score(all_actual_values, all_predicted_values)
print(f"R^2 general: {r2_general}")


# Guardar el conjunto de datos de prueba con las predicciones en un archivo CSV
test_data.to_csv('../assets/preciounidad_predicted.csv', index=False)

Merge results to a final dataset

In [47]:
predicted_partial_2023 = pd.read_csv('../assets/test.csv')
predicted_partial_2023 = predicted_partial_2023[['AÑO', 'MES', 'PRODUCTO']]

predicted_cantidadcompra = pd.read_csv('../assets/cantidadcompra_predicted.csv')
predicted_partial_2023['PREDICTED_CANTIDADCOMPRA'] = predicted_cantidadcompra['predicted']

predicted_preciounidad = pd.read_csv('../assets/preciounidad_predicted.csv')
predicted_partial_2023['PREDICTED_PRECIOUNIDAD'] = predicted_preciounidad['predicted']

predicted_partial_2023['PREDICTED_IMPORTELINEA'] = predicted_partial_2023['PREDICTED_CANTIDADCOMPRA'] * predicted_partial_2023['PREDICTED_PRECIOUNIDAD']

predicted_partial_2023.to_csv('../assets/predicted_partial_2023.csv', index=False)